# Исследование надёжности заёмщиков

В ходе исследования определим факторы влияющие на факт погашения кредита в срок. Ответим на следующие вопросы:

- Есть ли зависимость между количеством детей и возвратом кредита в срок?

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

- Как разные цели кредита влияют на его возврат в срок?

Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования необходимы для построения модели кредитного скоринга.

## Открытие файла с данными и изучение общей информации

In [1]:
#импорт библиотек
import pandas as pd 

In [2]:
data=pd.read_csv('C:\\Users\\web\\Desktop\\Progect\\data (1).csv') 

In [10]:
#выод первых 10 строк
display(data.head(10))
#вывод общ инф
display(data.info())
#describe
display(data.describe())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [9]:
# просмотр пропусков
pd.DataFrame(data.isna().mean()*100).style.set_precision(1).background_gradient('coolwarm')

,0
children,0.0
days_employed,10.1
dob_years,0.0
education,0.0
education_id,0.0
family_status,0.0
family_status_id,0.0
gender,0.0
income_type,0.0
debt,0.0


**Вывод**

- Пропуски в колонках 'days_employed', 'total_income'

- days_employed отрицательные значения

## Предобработка данных

In [11]:
#используя метод трансформ-эта функция в которой передаваемой функции возвращается то же выражение
avg_income=data.groupby('income_type')['total_income'].transform('median')
data['total_income'].fillna(avg_income,inplace=True)


In [12]:
#в столбце days_employed есть значения со знаком '-',обработаем их методом абс
data['days_employed']=abs(data['days_employed'])

In [13]:
# групируем данные income_type и days_employed и присваевыем медиану
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

In [14]:
#выводим уникальные значения детей
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Нужно удалить 2 аномалии 

возраст 20

возраст -1

In [15]:
#проверяем эти значения
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [17]:
#передаем в датафрейм условие в котором нет аномальных значений
data=data.loc[(data['children']>=0)&(data['children']<10)]
# проверка
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Удаление пропусков в days_employed

In [18]:
# воспользуемся тем же методом transform('median')
avg_days=data.groupby('income_type')['days_employed'].transform('median')
data['days_employed'].fillna(avg_days,inplace=True)# ваш код здесь

In [19]:
# проверяем тем же методом что и в задании №4
data.isna().sum()
#от пропусков избавились

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Изменения типов данных**

In [20]:
# заменяем тип данных в столбце total_income на целочисленные
data['total_income']=data['total_income'].astype('int')

**Обработка дубликатов**

In [21]:
# применяем метод к str.lower()к столбцу education
data['education']=data['education'].str.lower()

In [22]:
# считаем дубликаты и их кол-во методом duplicated().sum()
data.duplicated().sum()

71

In [23]:
data=data.drop_duplicates()

**Вывод**

- Избавились от пропусков в 'days_employed', 'total_income'

- Обработали дубликаты

- Обработали аномалии

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [24]:
# функция
def categorize_income(income):
    try:
        if 0<=income<=30000:
            return 'E'
        elif 30001<=income<=50000:
            return 'D'
        elif 50001<=income<=200000:
            return 'C'
        elif 200001<=income<=1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        'ошибка'

In [25]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [29]:
data['total_income_category'].value_counts()

C    15921
B     5014
D      349
A       25
E       22
Name: total_income_category, dtype: int64

Посмотрим уникальные значения целей для взятии кредита


In [30]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [31]:
def categorize_purpose(row):
    try:
        if 'авто' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образ' in row:
            return 'получение образования'
    except:
        'безкатегории'# создайте функцию categorize_purpose()

In [33]:
data['purpose_category']=data['purpose'].apply(categorize_purpose)
data['purpose_category'].value_counts()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

**Посмотрим,имеем ли зависимость между количеством детей и возвратом кредита в срок**

In [34]:
#формируем содную таблицу
children_pivot=data.pivot_table(index='children',values='debt',aggfunc=['count','sum','mean'])
print(children_pivot)

          count   sum      mean
           debt  debt      debt
children                       
0         14091  1063  0.075438
1          4808   444  0.092346
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000


**Вывод**

Исходя из представленных данных видно,что самые недобросовестные заемщики это люди с 2или1 ребенком(9.4% и 9.2% соответсвенно),можно было сказать что люди с 4 детьми самые злостные(9.7%),но количество таких случаев составляет 41шт(мене 1% от общ числа). опираться на такие данные не стоит,требуется больше информации.В целом по 'многодетным' трудно судить,так как котострафически мало случаев.

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [35]:
#cоздадим такую же сводную таблицу
family_pivot=data.pivot_table(index='family_status',values='debt',aggfunc=['count','sum','mean'])
print(family_pivot)

                       count  sum      mean
                        debt debt      debt
family_status                              
Не женат / не замужем   2796  273  0.097639
в разводе               1189   84  0.070648
вдовец / вдова           951   63  0.066246
гражданский брак        4134  385  0.093130
женат / замужем        12261  927  0.075606


**Вывод**

Да,есть!Исходя из представленных данных видно,что самые ответственные заемщики -это вдовцы/вдовы(6.6%),люди котоыре имели брак(7%) или находятся в нем(7.5%) меенее склонны к невозврату нежели оставшиеся группы.

**Как разные цели кредита влияют на его возврат в срок?**

In [36]:
purpose_pivot=data.pivot_table(index='purpose_category',values='debt',aggfunc=['count','sum','mean'])
print(purpose_pivot)

                          count  sum      mean
                           debt debt      debt
purpose_category                              
операции с автомобилем     4279  400  0.093480
операции с недвижимостью  10751  780  0.072551
получение образования      3988  369  0.092528
проведение свадьбы         2313  183  0.079118


Проведение свадеб и операции с недвижимостью имеют самый низкий процент не возврата(7,9% и 7,2%) в отношении с другими группами операции с автомобилем,получения образования(9,3% и 9,2% соответсвено)

## Вывод

Для изучение представлленого датафрейма был использован язык програмирования Python и библиотека Pandas.При использовании данных интсрументов в данном датафрейме были обнаружены пропуски,вбросы и аномальные значения,которые в последствии были успешно устранены.Исходя из полученных результатов рекомендую заказчику принять к сведению следущющий вывод:**В целом можно сказать,что люди имеющие 2-1 ребенка,состоящие в гражданском браке,имея доход от 50.000-200.000 и цель кредита на автомобиль или обрзование-Больше всего склоны не возвращать долги!Напротив же люди не имеющиее детей,имеющие статус семейного положения(вдовец/вдова) c зарплатой от 200.000 - 1.000.000 скорее всего вернуть кредит.НО!Не обязательно что бы все факторы совпали.В расчетах видно,что люди имееющие 5 детей - не имееют задолжностей(возможно они и не брали кредит,что думаю маловероятно)**